$${\displaystyle{
\begin{aligned} min_{Q} D_{\text{KL}}(Q_{t-1}\|Q_{t}) - |Q| \newline
D_{\text{KL}}(Q_{t-1}\|Q_{t}) &= \sum _{i}Q(i) \ln \left({\frac {Q(i)}{Q_{t-1}(i)}}\right)
\end{aligned}}}
Q$$

## Un ejemplo de comparación entre embeddings

### Instalamos cosas necesarias e importamos

In [1]:
!python -m spacy download es
!pip install unidecode

    100% |████████████████████████████████| 36.7MB 97.2MB/s 
  Running setup.py install for es-core-news-sm ... - \ | done

    Linking successful
    /usr/local/lib/python3.6/dist-packages/es_core_news_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/es

    You can now load the model via spacy.load('es')

    100% |████████████████████████████████| 245kB 7.2MB/s 


In [0]:
# numpy para algebra, spacy para NLP (natural languaje processing)
import numpy as np
import spacy

#### Definimos un par de ejemplo

In [0]:
musico = "el video muestra una escena en una playa, se escucha la gente riendo y jugando. El mar está tranquilo, no se escuchan muchas olas, y el cielo despejado. Hay una pareja con un chico, que se alejó para jugar con una pelotita. Hay un grupo de adolescentes lejos hacia la derecha donde uno toca la guitarra."
fotografo = "Se ve un día soleado de verano en una playa, con vistas al horizonte. Hay una pareja acostada sobre una manta verde y violeta de figuras geométricas, y varias sombrillas celestes a la izquierda. A lo lejos se ve el mar, con agua bastante clara. El hombre de la pareja tiene un bigote y aparenta unos 45 años, la mujer está de espaldas."

In [0]:
# Cargamos la data del modulo para espaciol
nlp = spacy.load('es')

In [0]:
# parseamos los strings y los metemos en un objeto tipo NLP de spacy
musico = nlp(musico)
fotografo = nlp(fotografo)

In [6]:
# Este objeto ya trae muchos atributos utiles. Asi accedemos a las priemras palabras
musico[0], fotografo[0]

(el, Se)

In [0]:
# Calcular distancia coseno entre vectores
def similitud_coseno(x, y):
    return x @ y / (np.linalg.norm(x) * np.linalg.norm(y))

In [8]:
# en musico.vector tenemos el word embedding para ese ejempo
similitud_coseno(musico.vector, fotografo.vector)

0.88435036

In [9]:
# unidecode para sacar tildes, nltk es otra para NLP
import unidecode
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
# descargamos una lista de palabras que no aportan (articulos, etc)
STOP_WORDS = set(stopwords.words('spanish'))

In [0]:
# ahora hacemos una version del ejemplo mas limpia: pasamos todo a minuscula, aplicamos lemmatizacion (ej: de corriamos a correr), y sacamos tildes
musico_clean = nlp(' '.join([unidecode.unidecode(w.string.strip()) for w in musico if w.lemma_.lower() not in STOP_WORDS]))
fotografo_clean = nlp(' '.join([unidecode.unidecode(w.string.strip()) for w in fotografo if w.lemma_.lower() not in STOP_WORDS]))

In [12]:
musico_clean, fotografo_clean

(video muestra escena playa , escucha gente riendo jugando . mar tranquilo , escuchan olas , cielo despejado . pareja chico , alejo para jugar pelotita . grupo adolescentes lejos hacia derecha uno toca guitarra .,
 ve dia soleado verano playa , vistas horizonte . pareja acostada sobre manta verde violeta figuras geometricas , varias sombrillas celestes izquierda . lejos ve mar , agua bastante clara . hombre pareja tiene bigote aparenta 45 anos , mujer espaldas .)

In [13]:
nlp('música').similarity(nlp('musica'))

0.7539651379961275

In [14]:
similitud_coseno(nlp('musica').vector, nlp('música').vector)

0.7539652

In [15]:
o1, o2 = 'escuchar sonido violín orquesta', 'miraba reflejo color luna oscuro'

e1, e2 = nlp(o1).vector, nlp(o2).vector
r1 = nlp('música, sonido, melodía, armonía, grave, agudo').vector 
r2 = nlp('fotografía, imagen, color, forma, contorno, linea').vector

print('|e1 - r1|', similitud_coseno(e1, r1))
print('|e2 - r1|', similitud_coseno(e2, r1))
print('|e1 - r2|', similitud_coseno(e1, r2))
print('|e2 - r2|', similitud_coseno(e2, r2))

|e1 - r1| 0.3672547
|e2 - r1| 0.4227681
|e1 - r2| 0.33383876
|e2 - r2| 0.35960206


Nos dan bastante parecido. Tiene sentido, porque ambos hablan de la playa. En este caso se calculo un embedding para todo el texto, eso no es conveniente. 1) separar por oracion. Ahora, es probable que eso tampoco nos de suficiente detalle, ya que queremos saber cuanto hablan de cada dominio. Creo mejor usar los embeddings por palabra, donde tendriamos informacion sobre cada palabra que tan relacionada está con cada dominio, y podemos manualmente sumar esas dimensiones para ver un puntaje por dominio para cada descripcion. Vamos a tener una distribucion de probabilidad para esos puntajes, y con eso podemos hacer una prueba T entre grupos: musicos contra fotografos, actores contra matematicos, etc.

In [36]:
nlp.vocab

In [20]:
e1, e2 = np.random.choice(len(nlp.vocab), size = 2)
print(nlp.vocab[e1],nlp.vocab[e2])

KeyError: ignored